In [10]:
import pandas as pd
import pyarrow as pa

Lendo o arquivo e fazendo a análise exploratória com a função describe()

In [11]:
df_california = pd.read_csv('california_housing_train.csv', sep=',')
df_california.dropna(inplace = True)
desc = df_california.describe()

QUESTÃO 1

In [12]:
#Questão 1.1
#usando a transposta para definir e agrupar qual a coluna com o maior desvio padrão
desc=desc.T
print("Desvio padrão do maior para o menor:\n",desc['std'].sort_values(ascending=False))
#Questão 1.2
#Encontrando o valor máximo e mínimo da coluna median_house_value
print("o maior valor de median_house_value é:",df_california['median_house_value'].max(),"\n")
print("o menor valor de median_house_value é:",df_california['median_house_value'].min(),"\n")

Desvio padrão do maior para o menor:
 median_house_value    115983.764387
total_rooms             2179.947071
population              1147.852959
total_bedrooms           421.499452
households               384.520841
housing_median_age        12.586937
latitude                   2.137340
longitude                  2.005166
median_income              1.908157
Name: std, dtype: float64
o maior valor de median_house_value é: 500001.0 

o menor valor de median_house_value é: 14999.0 



QUESTÃO 2

In [13]:
#Questão 2.1
#Criando os filtros de faixa etáriadados na questão
filtro_ate_18= (df_california['housing_median_age'] <18)
filtro_ate_29= (df_california['housing_median_age']>=18) & (df_california['housing_median_age']<29)
filtro_ate_37= (df_california['housing_median_age']>=29) & (df_california['housing_median_age']<37)
filtro_acima_37= (df_california['housing_median_age'] >37)

In [14]:
#Atribuindo os valores as linhas de acordo com os filtros de faixa etária
df_california.loc[filtro_ate_18, 'hma_cat'] = 'de_0_ate_18'
df_california.loc[filtro_ate_29, 'hma_cat'] = 'ate_29'
df_california.loc[filtro_ate_37, 'hma_cat'] = 'ate_37'
df_california.loc[filtro_acima_37, 'hma_cat'] = 'acima_37'

In [15]:
#Questão 2.2
#Atribuindo os valores as linhas de acordo com os filtros de regiã
df_california.loc[df_california['longitude']<-119, 'c_ns']='norte'
df_california.loc[df_california['longitude']>=-119, 'c_ns']='sul'

In [16]:
#Questão 2.3
#Renomeando as colunas para o padrão informado na questão e criando o arquivo .parquet
df_california.rename(columns={'hma_cat': 'age', 'c_ns':'california_region'}, inplace = True)
df_california[['age','california_region','total_rooms','total_bedrooms','population','households','median_house_value']]\
    .to_parquet('california_housing_train.parquet')
df_final = pd.read_parquet('california_housing_train.parquet')


In [17]:
#Questão 3
#Separando as colunas pedidas
df_final = df_final[['age','california_region','population','median_house_value']]

#Usando o grupby para fazer a agregação dos dados utilizando a função sum() e mean()
df_final_sum =df_final.groupby(['age','california_region'],as_index=False).sum()
df_final_mean=df_final.groupby(['age','california_region'], as_index=False).mean()

#Imprimindo na tela as tabelas dos resultados das funções sum() e mean()
print('Tabela com a média dos valores das casas por região e idade: \n',df_final_mean,"\n")
print('Tabela com a soma da população por região e idade:\n',df_final_sum,"\n")

#Definindo a dataframe final com base nos dataframes df_final_sum e df_final_mean
df= df_final_sum
df['median_house_value']=df_final_mean['median_house_value']
df.sort_values(by='median_house_value')
df.rename(columns={'population': 's_population', 'median_house_value': 'm_median_hpuse_value'}, inplace=True)

#Criando um arquivo .parquet com o resultado final da análise
df = pd.read_parquet('california_housing_train_analises.parquet')

Tabela com a média dos valores das casas por região e idade: 
            age california_region   population  median_house_value
0     acima_37             norte   967.558882       218114.239521
1     acima_37               sul  1090.287961       230514.625553
2       ate_29             norte  1395.961815       188724.023810
3       ate_29               sul  1716.760440       220571.658462
4       ate_37             norte  1153.022508       195766.603215
5       ate_37               sul  1356.746445       212266.993286
6  de_0_ate_18             norte  1681.368123       177826.697686
7  de_0_ate_18               sul  2088.391261       209407.565043 

Tabela com a soma da população por região e idade:
            age california_region  population  median_house_value
0     acima_37             norte   1938988.0         437100936.0
1     acima_37               sul   2218736.0         469097263.0
2       ate_29             norte   3107411.0         420099677.0
3       ate_29               

FileNotFoundError: [Errno 2] No such file or directory: 'california_housing_train_analises.parquet'

In [ ]:
df